# RFM-анализ и сегментация клиентов  

Для оптимизации маркетинговых и продающих стратегий компании стремятся к лучшему пониманию своих клиентов.  

Для этого мы используем RFM-анализ, который позволит нам сегментировать клиентов на основе их покупательского поведения и выявить наиболее ценные группы.  

Это позволит компаниям разрабатывать персонализированные маркетинговые стратегии, нацеленные на конкретные сегменты клиентов, что, в свою очередь, повысит эффективность продаж и лояльность клиентов.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
file_path = '/content/drive/My Drive/csv/RFM_ht_data.csv'
orders = pd.read_csv(file_path, parse_dates=['InvoiceDate'], low_memory=False)


Знакомимся с данными.

In [ ]:
orders.head()

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00


В данной таблице присутствует такая информация как:  
InvoiceNo: Номер заказа.  
CustomerCode: Идентификатор клиента (покупателя) который совершил заказ.   
InvoiceDate: Дата покупки    
Amount: Сумма покупок



In [ ]:
# смотрим размерность таблицы,сколько строк
orders.shape[0]

332730

In [ ]:
# смотрим пропущенные значения
orders.isna().mean()

,0
InvoiceNo,0.0
CustomerCode,0.0
InvoiceDate,0.0
Amount,0.0


In [ ]:
# Приводим данные в нужный формат для дальнейшей работы
orders['InvoiceDate'] = pd.to_datetime(orders['InvoiceDate'])
orders['CustomerCode'] = orders['CustomerCode'].apply(str)
orders['InvoiceNo'] = orders['InvoiceNo'].apply(str)

In [ ]:
# здесь получаем информацию о типах значений
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332730 entries, 0 to 332729
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   InvoiceNo     332730 non-null  object        
 1   CustomerCode  332730 non-null  object        
 2   InvoiceDate   332730 non-null  datetime64[ns]
 3   Amount        332730 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 10.2+ MB


In [ ]:
orders['CustomerCode'].apply(type).value_counts()

,count
CustomerCode,
<class 'str'>,332730


Для понимания масштаба клиентской базы мы идентифицируем количество уникальных клиентов.

In [ ]:
orders['CustomerCode'].nunique()

123733

Проанализируем временной диапазон, в котором были совершены покупки.

In [ ]:
orders['InvoiceDate'].min()

Timestamp('2020-09-01 00:00:00')

In [ ]:
orders['InvoiceDate'].max()

Timestamp('2020-09-30 00:00:00')

In [ ]:
orders['InvoiceDate'].describe()

,InvoiceDate
count,332730
mean,2020-09-15 16:35:03.846361856
min,2020-09-01 00:00:00
25%,2020-09-08 00:00:00
50%,2020-09-16 00:00:00
75%,2020-09-24 00:00:00
max,2020-09-30 00:00:00


### RFM-анализ и сегментация клиентов

Мы определим RFM-показатели для каждого клиента, а именно:

Recency (давность): Количество дней, прошедших с момента последней покупки клиента.  
Frequency (частота): Общее количество заказов, совершенных клиентом за рассматриваемый период.  
Monetary (денежная ценность): Суммарная стоимость всех заказов, сделанных клиентом.  

In [ ]:
last_date = orders['InvoiceDate'].max()
last_date

Timestamp('2020-09-30 00:00:00')

In [ ]:
# сводная таблица rfm
rfmTable = orders.groupby('CustomerCode').agg({'InvoiceDate': lambda x: (last_date - x.max()).days, # Recency #Количество дней с последнего заказа
                                        'InvoiceNo': lambda x: len(x),      # Frequency #Количество заказов
                                        'Amount': lambda x: x.sum()}) # Monetary Value #Общая сумма по всем заказам


In [ ]:
# сбрасываем индекс таблицы и преобразуем значения в целочисленные
int(rfmTable.reset_index()['CustomerCode'].iloc[0])

2213019

In [ ]:
rfmTable.head()

,InvoiceDate,InvoiceNo,Amount
CustomerCode,,,
02213019,19,1,1609.20
02213042,22,3,9685.48
02213071,29,1,415.00
02213088,23,1,305.00
02213092,25,1,1412.88


In [ ]:
# убедимся что размерность таблицы совпвдает с уникальными значениями
rfmTable.info()

<class 'pandas.core.frame.DataFrame'>
Index: 123733 entries, 02213019 to 99099972
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceDate  123733 non-null  int64  
 1   InvoiceNo    123733 non-null  int64  
 2   Amount       123733 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 3.8+ MB


In [ ]:
# для удобства восприятия можем переименовать столбцы
rfmTable.rename(columns={'InvoiceDate': 'recency',
                         'InvoiceNo': 'frequency',
                         'Amount': 'monetary_value'}, inplace=True)

Проведем сегментацию клиентов, разделив их на группы на основе RFM-значений и используя квантили (25%, 50%, 75%) для определения границ каждого сегмента.

In [ ]:
rfmSegmentation = rfmTable

In [ ]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])

In [ ]:
quantiles

,recency,frequency,monetary_value
0.25,2.0,1.0,765.00
0.50,8.0,2.0,1834.48
0.75,16.0,3.0,4008.84


Для сегментации клиентов мы присвоим каждому клиенту ранг на основе его значений Recency, Frequency и Monetary, используя следующую систему классификации:  

Recency (давность): Чем меньше значение (время с последней покупки), тем выше присваиваемый класс (1, 2, 3 или 4).  
Frequency (частота): Чем больше значение (количество покупок), тем выше присваиваемый класс (1, 2, 3 или 4).  
Monetary (денежная ценность): Чем больше значение (общая сумма покупок), тем выше присваиваемый класс (1, 2, 3 или 4).  


In [ ]:
def RClass(value,parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 1
    elif value <= quantiles_table[parameter_name][0.50]:
        return 2
    elif value <= quantiles_table[parameter_name][0.75]:
        return 3
    else:
        return 4

def FMClass(value, parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 4
    elif value <= quantiles_table[parameter_name][0.50]:
        return 3
    elif value <= quantiles_table[parameter_name][0.75]:
        return 2
    else:
        return 1

In [ ]:
# создаем новые столбцы на основе ранговых значений которые определяет соответствующая функция
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles)) # args нужные параметры

rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles))

rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles))

# создаем столбец который объединяет значения классов Recency, Frequency и Monetary в одну строку
rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) \
                            + rfmSegmentation.F_Quartile.map(str) \
                            + rfmSegmentation.M_Quartile.map(str)

In [ ]:
# результат
rfmSegmentation.head()

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
CustomerCode,,,,,,,
02213019,19,1,1609.20,4,4,3,443
02213042,22,3,9685.48,4,2,1,421
02213071,29,1,415.00,4,4,4,444
02213088,23,1,305.00,4,4,4,444
02213092,25,1,1412.88,4,4,3,443


In [ ]:
# перепроверяем правильна ли размерность таблицы
rfmSegmentation.shape[0]

123733

In [ ]:
rfmSegmentation['frequency'].max()

204

In [ ]:
rfmSegmentation['recency'].max()

29

Теперь мы можем узнать какое количество клиентов находится в тех сегментах,которые нас интересуют.

In [ ]:
rfmSegmentation[rfmSegmentation['RFMClass'] == '111'].shape[0]

9705

In [ ]:
rfmSegmentation[rfmSegmentation['RFMClass'] == '311'].shape[0]

1609

In [ ]:
# можем отсортировать клиентов,для того что бы определить в каком сегменте находится самое большое их количество
rfmSegmentation.groupby('RFMClass').size().sort_values(ascending = False)

,0
RFMClass,
444,10624
111,9705
443,6729
344,6593
211,5847
...,...
424,63
214,60
114,60
